# 03/11/24 - Debugging the JAM method with spherical geometry and with OM anisotropy.
### 02/22/24 - Adding covariance matrix to the space_jam method
### 02/12/24 - Adding least squares fitting option to forgo the time consuming MCMC for testing purposes.
### 12/26/23 - This notebook tests the modules "space_jam" and "total_mass_mge" in e.g. home/shawnknabel/Documents/slacs_kinematics/my_python_packages/space_jam.py

In [1]:
# import general libraries and modules
import numpy as np
np.set_printoptions(threshold=10000)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 6)
#plt.switch_backend('agg')
%matplotlib inline
import pandas as pd
import warnings
warnings.filterwarnings( "ignore", module = "matplotlib\..*" )
warnings.filterwarnings( "ignore", module = "plotbin\..*" )
import os
from os import path
from pathlib import Path
#import pickle
import dill as pickle
from datetime import datetime
def tick():
    return datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
import glob

# astronomy/scipy
from astropy.io import fits
#from astropy.wcs import WCS
#from scipy.ndimage import rotate
#from scipy.ndimage import map_coordinates
#from scipy.optimize import least_squares as lsq
#from astropy.convolution import convolve, convolve_fft, Gaussian2DKernel
#from astropy.cosmology import Planck15 as cosmo # I took 15 because for some reason Planck18 isn't in this astropy install #Planck18 as cosmo  # Planck 2018
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Tcmb0=2.725)
#from scipy.interpolate import interp1d
#from scipy.optimize import fsolve
import astropy.units as u
import astropy.constants as constants

# my functions
import sys
sys.path.append("/home/shawnknabel/Documents/slacs_kinematics/my_python_packages")


################################################################
# some needed information
kcwi_scale = 0.1457  # arcsec/pixel
hst_scale = 0.050 # ACS/WFC

# value of c^2 / 4 pi G
c2_4piG = (constants.c **2 / constants.G / 4 / np.pi).to('solMass/pc')


In [2]:
# bring in the space_jam and total_mass_mge modules

from space_jam import space_jam
from total_mass_mge import total_mass_mge

In [3]:
##################################################################################################################################

old_date_of_kin = '2023-02-28_2'
date_of_kin = '2024_02_15'

#------------------------------------------------------------------------------
# Directories and files

# data directory
data_dir = '/data/raw_data/KECK_KCWI_SLACS_kinematics_shawn/'
hst_dir = '/data/raw_data/HST_SLACS_ACS/kcwi_kinematics_lenses/'
tables_dir = f'{data_dir}tables/'
mosaics_dir = f'{data_dir}mosaics/'
kinematics_full_dir = f'{data_dir}kinematics/'
kinematics_dir =f'{kinematics_full_dir}{date_of_kin}/'
old_kinematics_dir = f'{kinematics_full_dir}{old_date_of_kin}/'
jam_output_dir = f'{data_dir}jam_outputs/'
# create a directory for JAM outputs
#Path(jam_output_dir).mkdir(parents=True, exist_ok=True)
#print(f'Outputs will be in {jam_output_dir}')
print()

# target SN for voronoi binning
vorbin_SN_targets = np.array([10, 15, 20])

#################################################
# objects
obj_names = ['SDSSJ0029-0055', 
             'SDSSJ0037-0942',
             'SDSSJ0330-0020',
             'SDSSJ1112+0826',
             'SDSSJ1204+0358',
             'SDSSJ1250+0523',
             'SDSSJ1306+0600',
             'SDSSJ1402+6321',
             'SDSSJ1531-0105',
             'SDSSJ1538+5817',
             'SDSSJ1621+3931',
             'SDSSJ1627-0053',
             'SDSSJ1630+4520',
             'SDSSJ2303+1422'
            ]

#################################################

paper_table = pd.read_csv(f'{tables_dir}paper_table_100223.csv')
slacs_ix_table = pd.read_csv(f'{tables_dir}slacs_ix_table3.csv')
zs = paper_table['zlens']
zlenses = slacs_ix_table['z_lens']
zsources = slacs_ix_table['z_src']
# get the revised KCWI sigmapsf
sigmapsf_table = pd.read_csv(f'{tables_dir}kcwi_sigmapsf_estimates.csv')

# 03/11/24 - Spherical Geometry is not working. Here, I am going to debug. 
## There are only 5 parameters. Let's roll.
## Use lsq minimization

In [12]:
'''
Run spherical geometry
'''

def space_jam_sph(i):

    obj_name = obj_names[i]
    kin_dir = f'{kinematics_dir}{obj_name}/'
    jam_dir = f'{jam_output_dir}{obj_name}/'
    jam_details_file = f'{kin_dir}{obj_name}_details_for_jampy_xshooter_{date_of_kin}.pkl'
    mass_model='power_law'
    anisotropy='const'
    geometry='sph'
    align='sph'
    zlens= zlenses[slacs_ix_table['Name']==obj_name].to_numpy()[0]
    zsource = zsources[slacs_ix_table['Name']==obj_name].to_numpy()[0]
    sigmapsf = sigmapsf_table[sigmapsf_table['obj_name']==obj_name]['kcwi_sigmapsf'].to_numpy()[0]
    #cosmo = cosmo
    fast_slow = paper_table.loc[0, 'class_for_JAM_models']
    systematics_est = 0.05 # %
    covariance_est = 0.02 # %
    p0 = [2.0, 0.9, 1.0, 0.5, 7] # gamma0, k_ani, einstein radius, k_mst, a_mst
    bounds = [[1.4, 0.0, 0.7, 0.0, 5 ], 
              [2.8, 1.0, 2.0, 1.0, 10]]
    sigpar = np.array([0.2, 0.2, 0.2, 0.2, 0.2])  # crude estimate of uncertainties
    prior_type = ['uniform','uniform','uniform','uniform','uniform']
    lensprior = False
    fix_pars = [0, 0, 0, 1, 1]
    lambda_int= 1.0
    nstep = 1000
    nwalkers = 12
    ndim = 5
    minimization = 'lsq'
    sampler_args = [nstep, nwalkers, ndim] # 10 walkers
    date_time = datetime.now().strftime("%Y_%m_%d")#-%I_%M_%S_%p"
    run_id = 1

    welcome_to_the_jam = space_jam(jam_dir, jam_details_file,
                                     obj_name, mass_model, anisotropy, geometry, align, 
                                    zlens=zlens, zsource=zsource, cosmo=cosmo, sigmapsf=sigmapsf, fast_slow=fast_slow,
                                   systematics_est=systematics_est, covariance_est=covariance_est,
                                   p0=p0, bounds=bounds, sigpar=sigpar, prior_type=prior_type, lensprior=lensprior, fix_pars=fix_pars, lambda_int=lambda_int,
                                   minimization=minimization, sampler_args=sampler_args, date_time=date_time, run_id=run_id, plot=True, overwrite=True, 
                                   test_prior=False, constant_err=False, kinmap_test=None)

    welcome_to_the_jam.fit_lsq()
    #welcome_to_the_jam.summary_plot()
    #welcome_to_the_jam.save_space_jam()

# Run it on J0029 index 0

In [13]:
space_jam_sph(0)

Files in /data/raw_data/KECK_KCWI_SLACS_kinematics_shawn/jam_outputs/SDSSJ0029-0055/SDSSJ0029-0055_2024_03_12_v1/ will be overwritten.

JAM Outputs to  /data/raw_data/KECK_KCWI_SLACS_kinematics_shawn/jam_outputs/SDSSJ0029-0055/SDSSJ0029-0055_2024_03_12_v1/

[0.0193628  0.1271433  0.245625   0.62405075 1.096469   1.30303874
 2.41793371 4.86902665 4.86902665] [0.80435509 0.8        0.80871018 0.82702924 0.8        0.9
 0.8        0.8        0.9       ]
Priors are now  ['uniform', 'uniform', 'uniform', 'uniform', 'uniform']
Mean prior values are  [2.0, 0.9, 1.0, 0.5, 7]
Added 5.0% systematics to diagonal.
Covariance terms are 2.0%.
Residuals > 10%: Change `inner_slope` or `outer_slope` or increase `ngauss`
Residuals > 10%: Change `inner_slope` or `outer_slope` or increase `ngauss`
Residuals > 10%: Change `inner_slope` or `outer_slope` or increase `ngauss`
Residuals > 10%: Change `inner_slope` or `outer_slope` or increase `ngauss`

Start lambda: 98.16  chi2: 430.9
Start p_free: 2.000 0.900

# Now OM

In [15]:
'''
Run spherical geometry
'''

def space_jam_sph_OM(i):

    obj_name = obj_names[i]
    kin_dir = f'{kinematics_dir}{obj_name}/'
    jam_dir = f'{jam_output_dir}{obj_name}/'
    jam_details_file = f'{kin_dir}{obj_name}_details_for_jampy_xshooter_{date_of_kin}.pkl'
    mass_model='power_law'
    anisotropy='OM'
    geometry='sph'
    align='sph'
    zlens= zlenses[slacs_ix_table['Name']==obj_name].to_numpy()[0]
    zsource = zsources[slacs_ix_table['Name']==obj_name].to_numpy()[0]
    sigmapsf = sigmapsf_table[sigmapsf_table['obj_name']==obj_name]['kcwi_sigmapsf'].to_numpy()[0]
    #cosmo = cosmo
    fast_slow = paper_table.loc[0, 'class_for_JAM_models']
    systematics_est = 0.05 # %
    covariance_est = 0.02 # %
    p0 = [2.0, 0.9, 1.0, 0.5, 7] # gamma0, k_ani, einstein radius, k_mst, a_mst
    bounds = [[1.4, 0.0, 0.7, 0.0, 5 ], 
              [2.8, 1.0, 2.0, 1.0, 10]]
    sigpar = np.array([0.2, 0.2, 0.2, 0.2, 0.2])  # crude estimate of uncertainties
    prior_type = ['uniform','uniform','uniform','uniform','uniform']
    lensprior = False
    fix_pars = [0, 0, 0, 1, 1]
    lambda_int= 1.0
    nstep = 1000
    nwalkers = 12
    ndim = 5
    minimization = 'lsq'
    sampler_args = [nstep, nwalkers, ndim] # 10 walkers
    date_time = datetime.now().strftime("%Y_%m_%d")#-%I_%M_%S_%p"
    run_id = 1

    welcome_to_the_jam = space_jam(jam_dir, jam_details_file,
                                     obj_name, mass_model, anisotropy, geometry, align, 
                                    zlens=zlens, zsource=zsource, cosmo=cosmo, sigmapsf=sigmapsf, fast_slow=fast_slow,
                                   systematics_est=systematics_est, covariance_est=covariance_est,
                                   p0=p0, bounds=bounds, sigpar=sigpar, prior_type=prior_type, lensprior=lensprior, fix_pars=fix_pars, lambda_int=lambda_int,
                                   minimization=minimization, sampler_args=sampler_args, date_time=date_time, run_id=run_id, plot=True, overwrite=True, 
                                   test_prior=False, constant_err=False, kinmap_test=None)

    welcome_to_the_jam.fit_lsq()
    #welcome_to_the_jam.summary_plot()
    #welcome_to_the_jam.save_space_jam()

In [18]:
space_jam_sph_OM(0)

Files in /data/raw_data/KECK_KCWI_SLACS_kinematics_shawn/jam_outputs/SDSSJ0029-0055/SDSSJ0029-0055_2024_03_12_v1/ will be overwritten.

JAM Outputs to  /data/raw_data/KECK_KCWI_SLACS_kinematics_shawn/jam_outputs/SDSSJ0029-0055/SDSSJ0029-0055_2024_03_12_v1/

[0.0193628  0.1271433  0.245625   0.62405075 1.096469   1.30303874
 2.41793371 4.86902665 4.86902665] [0.80435509 0.8        0.80871018 0.82702924 0.8        0.9
 0.8        0.8        0.9       ]


UnboundLocalError: cannot access local variable 'bound_ani_ratio_hi' where it is not associated with a value

In [21]:
aaa = 'KCWI_PSJ0147_r_00014_icubes_sum.fits'

In [22]:
aaa[-18:-16]

'14'

In [31]:
file = '/data/raw_data/Keck_KCWI_drizzle/PSJ0147+4630/KCWI_PSJ0147_r_00010_icubes_sum.fits'

In [32]:
data = fits.getdata(file)
data.shape

(193, 46)

In [33]:
np.argmax(data)

6421

In [34]:
np.unravel_index(data.argmax(), data.shape)

(139, 27)

In [38]:
np.genfromtxt('/data/raw_data/Keck_KCWI_drizzle/PSJ0147+4630/config').astype(int)

array([[  19,   28,   70, 2270],
       [  26,   27,   69, 2288],
       [  38,   28,   67, 1982],
       [  45,   28,   70, 2679],
       [  17,   27,  135, 2560],
       [  28,   27,  131, 2286],
       [  36,   27,  130, 1762],
       [  44,   28,   70, 2536],
       [  39,   28,   67, 2138],
       [  18,   27,  135, 2496],
       [  27,   28,   69, 2143],
       [  37,   28,   68, 1491],
       [  16,   27,  135, 2257],
       [  29,   27,  131, 1833],
       [  15,   27,   70, 2224],
       [  34,   27,  130, 1960],
       [  24,   27,  138, 2623],
       [  35,   27,  130, 2054],
       [  14,   27,   70, 2174],
       [  25,   27,   69, 2338],
       [  12,   27,  138, 2105],
       [  33,   28,   68, 2605],
       [  40,   28,  136, 2171],
       [  23,   27,  138, 3025],
       [  32,   28,   68, 2389],
       [  13,   27,   70, 2063],
       [  22,   27,  138, 2518],
       [  41,   28,  136, 2214],
       [  21,   28,   70, 1885],
       [  42,   28,  136, 2967],
       [  